# Gộp bảng

In [1]:
import pandas as pd
san_pham=pd.read_csv('product_catalog_clean.csv')
dat_hang=pd.read_csv('order_details_clean.csv')
tra_lai=pd.read_csv('return_refund_clean.csv')

gop_don_hang=dat_hang.merge(san_pham,on='product_id',how='left')
gop_tra_hang = tra_lai.merge(san_pham, on="product_id", how="left")

# 1.Thống kê số lượng đơn theo danh mục.

In [2]:
don_theo_danh_muc=(
    gop_don_hang.groupby('category')['order_id']
    .count()
    .reset_index(name='so_luong_don')
)
print(don_theo_danh_muc)

   category  so_luong_don
0  gia dụng            54
1   điện tử            66


# 2.Tìm các đơn có quantity bất thường

In [3]:
q1,q3=gop_don_hang['quantity'].quantile([0.25,0.75])
iqr=q3-q1
don_bat_thuong=gop_don_hang[
    (gop_don_hang['quantity']<=0)|
    (gop_don_hang['quantity']>q3+1.5*iqr)
]
print('\n Đơn hàng bất thường')
print(don_bat_thuong)


 Đơn hàng bất thường
    order_id product_id  quantity  unit_price  order_date        product_name  \
11    OD3011       PR02        -1    250000.0  2024-03-13       loa bluetooth   
15    OD3015       PR03        -1    200000.0  2024-04-22         ban phim co   
22    OD3022       PR03        -1    200000.0  2024-01-01         ban phim co   
26    OD3026       PR02        -1    350000.0  2024-03-02       loa bluetooth   
27    OD3027       PR04        -1    250000.0  2024-03-23     chuot khong day   
45    OD3045       PR04        -1    350000.0  2024-04-04     chuot khong day   
80    OD3080       PR05        -1    250000.0  2024-04-09        sac du phong   
82    OD3082       PR01        -1    150000.0  2024-02-18  tai nghe bluetooth   
98    OD3098       PR02        -1    150000.0  2024-02-28       loa bluetooth   
104   OD3104       PR03        -1    350000.0  2024-03-14         ban phim co   
112   OD3112       PR03        -1   1250000.0  2024-04-05         ban phim co   
117   

# 3.Thống kê sản phẩm bán chạy nhất.

In [4]:
san_pham_ban_chay=(
    gop_don_hang.groupby(['product_id','product_name'])['quantity']
    .sum()
    .reset_index()
    .sort_values(by='quantity',ascending=False)
    .head(1)
)
print('\n Sản phẩm bán chạy nhất')
print(san_pham_ban_chay)


 Sản phẩm bán chạy nhất
  product_id  product_name  quantity
4       PR05  sac du phong        56


# 4.Tìm sản phẩm có nhiều đơn trả hàng nhất.

In [5]:
san_pham_tra_nhieu_nhat=(
    gop_tra_hang.groupby(['product_id','product_name'])
    .size()
    .reset_index(name='so_don_tra')
    .sort_values(by='so_don_tra',ascending=False)
)
print('\nSản phẩm có nhiều đơn trả hàng nhất')
print(san_pham_tra_nhieu_nhat)


Sản phẩm có nhiều đơn trả hàng nhất
  product_id        product_name  so_don_tra
3       PR04     chuot khong day          18
4       PR05        sac du phong          16
1       PR02       loa bluetooth          14
2       PR03         ban phim co           8
0       PR01  tai nghe bluetooth           4


# 5.Thống kê tỷ lệ trả hàng theo từng danh mục.

In [6]:
ty_le_tra_hang=(
    gop_tra_hang.groupby('category').size()
    /gop_don_hang.groupby('category').size()
).reset_index(name="ty_le_tra_hang")

print("\nTỶ LỆ TRẢ HÀNG THEO DANH MỤC")
print(ty_le_tra_hang)


TỶ LỆ TRẢ HÀNG THEO DANH MỤC
   category  ty_le_tra_hang
0  gia dụng        0.629630
1   điện tử        0.393939
